In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time
from pathlib import Path
import pandas as pd
import re

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
path_to_circuits = '../../circuits/qasm/'
input_data = {"Name": [], "circuit": [], "graph": []}

for file in Path(path_to_circuits).glob('*.qasm'):
    circuit = zx.Circuit.load(file)
    if circuit.qubits <= 20 and circuit.qubits >= 8 and len(circuit.gates) <= 5000 and len(circuit.gates) >= 100:
        try:
            circuit = zx.optimize.basic_optimization(circuit.split_phase_gates())
        except Exception as e:
            pass
        graph = circuit.to_graph()

        input_data["Name"].append(file.stem)
        input_data["circuit"].append(circuit)
        input_data["graph"].append(circuit.to_graph())

In [4]:
print(len(input_data["Name"]))

7


In [5]:
# Define the column names
columns = input_data["Name"]

# Define the row labels
rows = ["Gates", "T-Count", "Cliffords", "CNOTS", "Other 2 Qubit Gates", "Hadamard", "Time"]

#Define the algorithm
algorithm = ["TR", "FR", "G0", "G1", "G2", "G3", "GN"]
# algorithm = ["TR", "FR", "G0", "G1", "G2", "G3", "GN", "GSA", "GSAN"]

dataframes = []

In [6]:
def run_algorithm(algorithm, input_data, dataframes):
    
    gates = []
    t_count = []
    cliffords = []
    cnot = []
    other = []
    hadamard = []
    times = []

    for name, circuit, graph in zip(input_data["Name"], input_data["circuit"], input_data["graph"]):

        # if name == "ham15-high":

        start = time.perf_counter()
        graph_simplified = graph.clone()
        algorithm(graph_simplified)
        end = time.perf_counter() - start

        stats = zx.extract_circuit(graph_simplified.copy()).stats()
        # Extract the numbers
        numbers = re.findall(r'\d+', stats)

        # Assign the numbers to variables
        gates.append(int(numbers[1]))
        t_count.append(int(numbers[2]))
        cliffords.append(int(numbers[3]))
        cnot.append(int(numbers[6]))
        other.append(int(numbers[7]))
        hadamard.append(int(numbers[8]))
        times.append(int(end))

    data = [gates, t_count, cliffords, cnot, other, hadamard, times]
    dataframes.append(pd.DataFrame(data, columns=columns, index=rows))

In [7]:
run_algorithm(zx.simplify.teleport_reduce, input_data, dataframes)

In [8]:
run_algorithm(zx.simplify.full_reduce, input_data, dataframes)

In [9]:
from functools import partial

for la in range(4):
    partial_la = partial(zx.simplify.greedy_simp, lookahead=la)
    run_algorithm(partial_la, input_data, dataframes)

INFO:root:greedy_wire_reduce_count: 16


INFO:root:greedy_wire_reduce_count: 3
INFO:root:greedy_wire_reduce_count: 38
INFO:root:greedy_wire_reduce_count: 29
INFO:root:greedy_wire_reduce_count: 14
INFO:root:greedy_wire_reduce_count: 15
INFO:root:greedy_wire_reduce_count: 9
INFO:root:greedy_wire_reduce_count: 15
INFO:root:greedy_wire_reduce_count: 3
INFO:root:greedy_wire_reduce_count: 37
INFO:root:greedy_wire_reduce_count: 29
INFO:root:greedy_wire_reduce_count: 14
INFO:root:greedy_wire_reduce_count: 15
INFO:root:greedy_wire_reduce_count: 8
INFO:root:greedy_wire_reduce_count: 14
INFO:root:greedy_wire_reduce_count: 2
INFO:root:greedy_wire_reduce_count: 36
INFO:root:greedy_wire_reduce_count: 29
INFO:root:greedy_wire_reduce_count: 17
INFO:root:greedy_wire_reduce_count: 15
INFO:root:greedy_wire_reduce_count: 7
INFO:root:greedy_wire_reduce_count: 4
INFO:root:greedy_wire_reduce_count: 13
INFO:root:greedy_wire_reduce_count: 1
INFO:root:greedy_wire_reduce_count: 35
INFO:root:greedy_wire_reduce_count: 31
INFO:root:greedy_wire_reduce_coun

In [10]:
run_algorithm(zx.simplify.greedy_simp_neighbors, input_data, dataframes)

KeyboardInterrupt: 

In [ ]:
# run_algorithm(zx.simplify.sim_anneal_simp, input_data, dataframes)

In [ ]:
# run_algorithm(zx.simplify.sim_anneal_simp_neighbors, input_data, dataframes)

In [11]:
df = pd.concat(dataframes, axis=0, keys=algorithm)
df.to_csv('benchmark_greedy_la.csv')

C:\Users\wsajk\AppData\Local\Temp\ipykernel_139704\2286571105.py:1: FutureWarning: The behavior of pd.concat with len(keys) != len(objs) is deprecated. In a future version this will raise instead of truncating to the smaller of the two sequences
  df = pd.concat(dataframes, axis=0, keys=algorithm)
